title: Business Cashback  
author: Fabio Schmidt-Fischbach   
date: 2020-12-20   
region: EU   
link: https://docs.google.com/presentation/d/1Oaprmkl_hKrwnfYUUYq1GZK9puUvHaxmhIcgrpc7P-k/edit?usp=sharing    
summary: All business tiers on average break even within 6 months - business standard is hence a profitable tier in contrast to personal standard. We spent so far 750k on business cashback this year. These cashback payments contribute roughly 5% of our business standard users LTV. 92% of MAUs get less than 2 Euros per month. 2% of MAUs (3.5k) get more than 5 Euros per month.
tags: memberships, cashback, business, premium, ltv, cost, perks    

In [2]:
import pandas as pd
import numpy as np
import altair as alt

## How many more smart users do we need to compensate X amount of users dropping to normal standard? 

In [ ]:
""" 
with initial_product as ( 
select user_created, product_id, subscription_valid_from
from dbt.zrh_user_product where enter_reason = 'SIGNUP' 
),
x as ( 
select
        user_created, 
        subscription_valid_from,
        product_id,
        sum(value::float/100) as ltv 
from dbt.ucm_pnl 
inner join dbt.ucm_mapping using (label)
inner join initial_product using (user_created) 
where datediff('month', subscription_valid_from::date, TO_DATE(month, 'YYYY-MM')) <= 12 and subscription_valid_from>='2018-01-01' and subscription_valid_from < current_date - interval '12 months'
group by 1,2,3 
)

select 
        product_id,
        avg(ltv) 
from x 
group by 1
"""
FLEX_ACCOUNT_MONTHLY	42.161
BUSINESS_CARD	12.079
BLACK_CARD_MONTHLY	44.237
BUSINESS_BLACK	59.657
STANDARD	-14.207
METAL_CARD_MONTHLY	32.089

In [155]:
import numpy as np

# what's the avg. net margin of business standard after 12 mos?
# https://metabase-product.tech26.de/question/3534

## Jan 2020 cohort avg. pnl values
values = {
    "metal": 100,
    "smart": 36.5,  ## compute ( b you - b standard ) / 2 [bc of half the margin] + b standard
    "you": 59,
    "bstandard": 12,
    "standard": -14,
}

# what are business only x-sell figures? use jan 2021
bmetal = 0.0257
byou = 0.0086
bst = 0.1771
bsmart = 0.0144
metal = 0.0386
you = 0.0221
smart = 0.0275
standard_normal = 0.6543
flex = 0.0326

total = bmetal + byou + bst + bsmart + you + metal + smart + standard_normal + flex
probs = {
    "metal": bmetal / total,
    "smart": bsmart / total,
    "you": byou / total,
    "bstandard": bst / total,
}


def compute_xsellift(value_bsmart, value_standard, value_bstandard, probs):
    """
    standard_prob = % of b-standard that switches to personal standard
    value_bsmart = Avg. net PNL after 12 months for business smart
    value_standard = Avg. net PNL after 12 months for standard customer
    value_bstandard = Avg. net PNL after 12 months for business standard customer
    probs = Dictionary of baseline xsell values at kyc

    """
    df = pd.DataFrame(
        np.linspace(0, 0.2, 10), columns=["business_standard_to_personal"]
    ).reset_index()

    def compute_smart(standard_prob):
        """
        Computes implied xsell to smart.
        """
        # how much we loose per customer due to personal standard cannibalization
        loss = probs["bstandard"] * standard_prob * (value_bstandard - value_standard)

        # adjust new business standard size
        new_bstandard = probs["bstandard"] * (1 - standard_prob)

        # What % of bstandard needs to upgrade to smart to make it worthwhile?
        smart_prob = loss / ((value_bsmart - value_bstandard) * new_bstandard)

        if smart_prob > 1:
            return np.nan
        else:
            return probs["smart"] + smart_prob * new_bstandard

    df["implied_smart"] = df.apply(
        lambda x: compute_smart(x["business_standard_to_personal"]), axis=1
    )

    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X(
                "business_standard_to_personal:Q",
                axis=alt.Axis(
                    format="%",
                    title="% of business standard that go to standard personal",
                ),
            ),
            y=alt.Y(
                "implied_smart:Q",
                axis=alt.Axis(format="%", title="required % x-sell (bus. smart)"),
            ),
        )
        .properties(
            width=500,
            height=500,
            title="Break even point: xsell to b.smart vs loss to normal standard",
        )
    )
    return chart


compute_xsellift(values["smart"], values["standard"], values["bstandard"], probs)

alt.Chart(...)

In [157]:
def output_perc_upsell(value_bsmart, value_standard, value_bstandard, probs):
    """
    standard_prob = % of b-standard that switches to personal standard
    value_bsmart = Avg. net PNL after 12 months for business smart
    value_standard = Avg. net PNL after 12 months for standard customer
    value_bstandard = Avg. net PNL after 12 months for business standard customer
    probs = Dictionary of baseline xsell values at kyc

    """
    df = pd.DataFrame(
        np.linspace(0, 0.2, 10), columns=["business_standard_to_personal"]
    ).reset_index()

    def compute_smart(standard_prob):
        """
        Computes implied xsell to smart.
        """
        # how much we loose per customer due to personal standard cannibalization
        loss = probs["bstandard"] * standard_prob * (value_bstandard - value_standard)

        # adjust new business standard size
        new_bstandard = probs["bstandard"] * (1 - standard_prob)

        # What % of bstandard needs to upgrade to smart to make it worthwhile?
        smart_prob = loss / ((value_bsmart - value_bstandard) * new_bstandard)

        if smart_prob > 1:
            return np.nan
        else:
            return smart_prob * new_bstandard / probs["bstandard"]

    df["implied_smart"] = df.apply(
        lambda x: compute_smart(x["business_standard_to_personal"]), axis=1
    )

    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X(
                "business_standard_to_personal:Q",
                axis=alt.Axis(
                    format="%",
                    title="% of business standard that go to standard personal",
                ),
            ),
            y=alt.Y(
                "implied_smart:Q",
                axis=alt.Axis(
                    format="%",
                    title="% bus. standard that need to choose smart instead",
                ),
            ),
        )
        .properties(
            width=500,
            height=500,
            title="Break even point: % upsell to b.smart vs loss to normal standard",
        )
    )
    return chart


output_perc_upsell(values["smart"], values["standard"], values["bstandard"], probs)

alt.Chart(...)

In [158]:
def output_perc_upsell(value_bsmart, value_standard, value_bstandard, probs):
    """
    standard_prob = % of b-standard that switches to personal standard
    value_bsmart = Avg. net PNL after 12 months for business smart
    value_standard = Avg. net PNL after 12 months for standard customer
    value_bstandard = Avg. net PNL after 12 months for business standard customer
    probs = Dictionary of baseline xsell values at kyc

    """
    df = pd.DataFrame(
        np.linspace(0, 0.2, 10), columns=["business_standard_to_personal"]
    ).reset_index()

    def compute_smart(standard_prob):
        """
        Computes implied xsell to smart.
        """
        # how much we loose per customer due to personal standard cannibalization
        loss = probs["bstandard"] * standard_prob * (value_bstandard - value_standard)

        # adjust new business standard size
        new_bstandard = probs["bstandard"] * (1 - standard_prob)

        # What % of bstandard needs to upgrade to smart to make it worthwhile?
        smart_prob = loss / ((value_bsmart - value_bstandard) * new_bstandard)

        if smart_prob > 1:
            return np.nan
        else:
            return smart_prob * new_bstandard / probs["bstandard"]

    df["implied_smart"] = df.apply(
        lambda x: compute_smart(x["business_standard_to_personal"]), axis=1
    )

    chart = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X(
                "business_standard_to_personal:Q",
                axis=alt.Axis(
                    format="%",
                    title="% of business standard that go to standard personal",
                ),
            ),
            y=alt.Y(
                "implied_smart:Q",
                axis=alt.Axis(
                    format="%",
                    title="% bus. standard that need to choose smart instead",
                ),
            ),
        )
        .properties(
            width=500,
            height=500,
            title="Break even point: % upsell to b.smart vs loss to normal standard",
        )
    )
    return chart


output_perc_upsell(values["smart"], 0, values["bstandard"], probs)

alt.Chart(...)

# A world with perfect retention / just focus on fees 

In [4]:
smart = 4.90
you = 9.90
metal = 16.90 
standard = 0 

def compute
